In [1]:
import os
import time
import shutil

In [2]:
from jax_md import space
import jax.numpy as np
def displacement(Ra, Rb, **unused_kwargs):
    dR = Ra - Rb
    return dR
metric = space.metric(displacement)
metrix = space.map_product(metric)

In [3]:
def getscore(file):
    cnt=0
    score=0.0
    with open(file) as f:
        for i in f:
            #print(cnt,i)
            if cnt==14:
                xi=i.split()
                score=float(xi[1])
                break
            cnt=cnt+1
    return score

In [4]:
from biopandas.pdb import PandasPdb
def readlig(file):
    data = PandasPdb().read_pdb(file)
    df = data.df['HETATM']
    typ=df['atom_name']
    lx=df['x_coord']
    ly=df['y_coord']
    lz=df['z_coord']
    osym=df['element_symbol']
    otyp=[]
    opos=[]
    for i in range(len(typ)):
        otyp.append(typ[i])
        opos.append([lx[i],ly[i],lz[i]])
    
    nom=['CHA', 'CHB', 'CHC', 'CHD', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'CBD', 'CGD', 'O1D', 'O2D', 'NA', 'NB', 'NC', 'ND', 'FE']
    if len(otyp)!=43:
        return otyp,opos
#    if otyp[0]!=nom[0]:
    dic={}
    new_pos=[]
    new_typ=[]
    new_sym=[]
    for i in range(len(otyp)):
        dic[otyp[i]]=i
    for i in nom:
        j=dic[i]
        new_pos.append(opos[j])
        new_typ.append(otyp[j])
        new_sym.append(osym[j])
    return new_typ,new_pos,new_sym
    
    return otyp,opos,osym

def readrec(file):
    data = PandasPdb().read_pdb(file)
    df = data.df['ATOM']
    res=df['residue_name']
    typ=df['atom_name']
    lx=df['x_coord']
    ly=df['y_coord']
    lz=df['z_coord']
    sym=df['element_symbol']
    otyp=[]
    opos=[]
    ores=[]
    osym=[]
    for i in range(len(typ)):
        otyp.append(typ[i])
        opos.append([lx[i],ly[i],lz[i]])
        ores.append(res[i])
        osym.append(sym[i])
    return otyp,opos,ores,osym

In [5]:
def readrecx(file):
    data = PandasPdb().read_pdb(file)
    df = data.df['ATOM']
    #print(df)
    res=df['residue_name']
    resnum=df['residue_number']
    typ=df['atom_name']
    lx=df['x_coord']
    ly=df['y_coord']
    lz=df['z_coord']
    sym=df['element_symbol']
    otyp=[]
    opos=[]
    ores=[]
    osym=[]
    onum=[]
    for i in range(len(typ)):
        otyp.append(typ[i])
        opos.append([lx[i],ly[i],lz[i]])
        ores.append(res[i])
        osym.append(sym[i])
        onum.append(resnum[i])
    return otyp,opos,ores,osym,onum

In [6]:
dicr={'H':0.32,'C':0.77,'N':0.75,'O':0.73,'S':1.02,'FE':0.76}
def ckcrash(pres,ps,ppos,hs,hpos):
    #检查蛋白质和小分子关系，p代表蛋白质数据，h代表小分子数据
    #print('***')
    
    #生成原子半径
    pr=[]
    hr=[]
    for i in ps:
        if i not in dicr:
            print(i,dicr)
            return
        pr.append(dicr[i])
    for i in hs:
        if i not in dicr:
            print(i,dicr)
            return
        hr.append(dicr[i])
    
    
    pr=np.array(pr)
    hr=np.array(hr)
    ppos=np.array(ppos)
    hpos=np.array(hpos)
    disph=[]
    result='success'
    minfes=100.0
    spos=0
    
    #计算heme小分子的原子与蛋白质原子的距离
    for i in range(len(hs)):
        dis=metric(hpos[i],ppos)-(pr+hr[i])
        disph.append(np.min(dis))
        if hs[i]=='FE':
            for j in range(len(pr)):
                if ps[j]=='S': #and pres[j]=='CYS':
                    fes=dis[j]+pr[j]+hr[i]
                    #print(fes)
                    if fes<minfes:
                        minfes=fes
                        spos=j
            if minfes<1.93 or minfes>2.77:
                result='Fe_remote'

    for i in disph:
#         if result!='success':
#             break
        if i<0:
            result='crash'
            break
        if i>11:
            result='notin'
            break

    return disph,result,minfes,spos

In [7]:
def process(rfile,lfile):
    #检查蛋白质和小分子的具体关系，rfile代表蛋白质数据路径，lfile代表小分子数据路径
#     if scorel[tag]!=omscore[tag]:
#         print('err')
#         return
#     com='./TMalign '+oplist[tag]+' '+orlist[tag]+' -o TM_sup > outtm.txt'
#     #print(com)
#     os.system(com)
    ptyp,ppos,pres,psym=readrec(rfile)
    ltyp,lpos,lsym=readlig(lfile)
    mindis,result,minfes,spos=ckcrash(pres,psym,ppos,lsym,lpos)
    return result,minfes
    crashn=np.sum(mindis<0)
    return crash,minfes,spos
    #print(minfes,min(mindis))
    if result!='crash':
        return result,minfes,spos
    #检查具体的crash情况
    nl=[ 'CBA', 'CGA', 'O1A', 'O2A',  'CBB', 'CBC', 'CBD', 'CGD', 'O1D', 'O2D'] 
    mintag=0
    crashl=[]
    for i in range(len(mindis)):
        if mindis[i]<0:
            if ltyp[i] not in nl:
                mintag=1
            crashl.append(ltyp[i])
    if mintag==0:
        return 'rotate',minfes,spos
    if len(crashl)==1 and crashl[0]=='FE':
        return 'Fe_crash',minfes,spos
    return 'other',minfes,spos

In [8]:
rcra=[]
rfes=[]
for i in range(9):
    rcra.append([])
    rfes.append([])

In [9]:
for i in range(20):
    for j in range(9):
        if i==6:
            continue
        rp='./alfpdb/alf'+str(i)+'/xr'+str(j)+'mdtm.pdb'
        lp='./alfpdb/alf'+str(i)+'/xr'+str(j)+'mdlig.pdb'
        
        if os.path.exists(rp) and os.path.exists(lp):
            print(i,j,'*')
            result,minfes=process(rp,lp)
            rcra[j].append(result)
            rfes[j].append(minfes)
            print(i,j)

0 0 *
0 0
0 1 *
0 1
0 2 *
0 2
0 3 *
0 3
0 4 *
0 4
0 5 *
0 5
0 6 *
0 6
0 7 *
0 7
1 0 *
1 0
1 1 *
1 1
1 2 *
1 2
1 3 *
1 3
1 4 *
1 4
1 5 *
1 5
1 6 *
1 6
1 7 *
1 7
2 0 *
2 0
2 1 *
2 1
2 2 *
2 2
2 3 *
2 3
2 4 *
2 4
2 5 *
2 5
2 6 *
2 6
2 7 *
2 7
3 0 *
3 0
3 1 *
3 1
3 2 *
3 2
3 3 *
3 3
3 4 *
3 4
3 5 *
3 5
3 6 *
3 6
3 7 *
3 7
5 0 *
5 0
5 1 *
5 1
5 2 *
5 2
5 3 *
5 3
5 4 *
5 4
5 5 *
5 5
5 6 *
5 6
5 7 *
5 7
7 0 *
7 0
7 1 *
7 1
7 2 *
7 2
7 3 *
7 3
7 4 *
7 4
7 5 *
7 5
7 6 *
7 6
7 7 *
7 7
8 0 *
8 0
8 1 *
8 1
8 2 *
8 2
8 3 *
8 3
8 4 *
8 4
8 5 *
8 5
8 6 *
8 6
8 7 *
8 7
9 0 *
9 0
9 1 *
9 1
9 2 *
9 2
9 3 *
9 3
9 4 *
9 4
9 5 *
9 5
9 6 *
9 6
9 7 *
9 7
10 0 *
10 0
10 1 *
10 1
10 2 *
10 2
10 3 *
10 3
10 4 *
10 4
10 5 *
10 5
10 6 *
10 6
10 7 *
10 7
11 0 *
11 0
11 1 *
11 1
11 2 *
11 2
11 3 *
11 3
11 4 *
11 4
11 5 *
11 5
11 6 *
11 6
11 7 *
11 7
12 0 *
12 0
12 1 *
12 1
12 2 *
12 2
12 3 *
12 3
12 4 *
12 4
12 5 *
12 5
12 6 *
12 6
12 7 *
12 7
13 0 *
13 0
13 1 *
13 1
13 2 *
13 2
13 3 *
13 3
13 4 *
13 4
13 5 *
13 5


In [13]:
for i in range(8):
    sumr=[]
    lenr=0
    for j in rfes[i]:
        if j<5:
            sumr.append((j-2.38)**2)
            lenr+=1
    sumr=np.array(sumr)
    print(i,rcra[i].count('crash')/50,np.mean(sumr)**0.5)

0 0.14 0.51706016
1 0.14 0.51663804
2 0.02 0.27567396
3 0.02 0.27567396
4 0.0 0.26187828
5 0.02 0.27567396
6 0.02 0.25477582
7 0.0 0.40440473


In [11]:
for i in range(8):
    sumr=[]
    lenr=0
    print(i,rfes[i])
#     for j in rfes[i]:
#         if j<5:
#             sumr.append((j-2.38)**2)
#             lenr+=1
#     sumr=np.array(sumr)
#     print(i,rcra[i].count('crash')/50,np.mean(sumr)**0.5)

0 [Array(13.615196, dtype=float32), Array(1.6801265, dtype=float32), Array(2.710153, dtype=float32), Array(2.6968734, dtype=float32), Array(30.470173, dtype=float32), Array(1.7932255, dtype=float32), Array(2.1634696, dtype=float32), Array(1.7378638, dtype=float32), Array(1.9769369, dtype=float32), Array(1.8950274, dtype=float32), Array(2.541342, dtype=float32), Array(2.4819863, dtype=float32), Array(1.5891132, dtype=float32), Array(2.0678947, dtype=float32), Array(5.8702555, dtype=float32), Array(1.637042, dtype=float32), Array(1.6511892, dtype=float32)]
1 [Array(13.622652, dtype=float32), Array(1.6807597, dtype=float32), Array(2.7077205, dtype=float32), Array(2.6968734, dtype=float32), Array(30.470947, dtype=float32), Array(1.7932255, dtype=float32), Array(2.1658847, dtype=float32), Array(1.7378638, dtype=float32), Array(1.9769369, dtype=float32), Array(1.8960209, dtype=float32), Array(2.541151, dtype=float32), Array(2.4806447, dtype=float32), Array(1.5891132, dtype=float32), Array(2.

In [12]:
rcrax=[]
rfesx=[]
for i in range(9):
    rcrax.append([])
    rfesx.append([])

In [13]:
for i in range(20):
    for j in range(9):
        if i==6:
            continue
        rp='./alfpdb/alf'+str(i)+'/xr'+str(j)+'tm.pdb'
        lp='./alfpdb/alf'+str(i)+'/xr'+str(j)+'lig.pdb'
        
        if os.path.exists(rp) and os.path.exists(lp):
            print(i,j,'*')
            result,minfes=process(rp,lp)
            rcrax[j].append(result)
            rfesx[j].append(minfes)
            print(i,j)

0 0 *
0 0
0 1 *
0 1
0 2 *
0 2
0 3 *
0 3
0 4 *
0 4
0 5 *
0 5
0 6 *
0 6
0 7 *
0 7
1 0 *
1 0
1 1 *
1 1
1 2 *
1 2
1 3 *
1 3
1 4 *
1 4
1 5 *
1 5
1 6 *
1 6
1 7 *
1 7
2 0 *
2 0
2 1 *
2 1
2 2 *
2 2
2 3 *
2 3
2 4 *
2 4
2 5 *
2 5
2 6 *
2 6
2 7 *
2 7
3 0 *
3 0
3 1 *
3 1
3 2 *
3 2
3 3 *
3 3
3 4 *
3 4
3 5 *
3 5
3 6 *
3 6
3 7 *
3 7
5 0 *
5 0
5 1 *
5 1
5 2 *
5 2
5 3 *
5 3
5 4 *
5 4
5 5 *
5 5
5 6 *
5 6
5 7 *
5 7
7 0 *
7 0
7 1 *
7 1
7 2 *
7 2
7 3 *
7 3
7 4 *
7 4
7 5 *
7 5
7 6 *
7 6
7 7 *
7 7
8 0 *
8 0
8 1 *
8 1
8 2 *
8 2
8 3 *
8 3
8 4 *
8 4
8 5 *
8 5
8 6 *
8 6
8 7 *
8 7
9 0 *
9 0
9 1 *
9 1
9 2 *
9 2
9 3 *
9 3
9 4 *
9 4
9 5 *
9 5
9 6 *
9 6
9 7 *
9 7
10 0 *
10 0
10 1 *
10 1
10 2 *
10 2
10 3 *
10 3
10 4 *
10 4
10 5 *
10 5
10 6 *
10 6
10 7 *
10 7
11 0 *
11 0
11 1 *
11 1
11 2 *
11 2
11 3 *
11 3
11 4 *
11 4
11 5 *
11 5
11 6 *
11 6
11 7 *
11 7
12 0 *
12 0
12 1 *
12 1
12 2 *
12 2
12 3 *
12 3
12 4 *
12 4
12 5 *
12 5
12 6 *
12 6
12 7 *
12 7
13 0 *
13 0
13 1 *
13 1
13 2 *
13 2
13 3 *
13 3
13 4 *
13 4
13 5 *
13 5


In [14]:
for i in range(8):
    sumr=[]
    lenr=0
    print(i,rfesx[i])

0 [Array(13.756277, dtype=float32), Array(2.1728497, dtype=float32), Array(2.7608593, dtype=float32), Array(2.3841074, dtype=float32), Array(30.37624, dtype=float32), Array(2.2532291, dtype=float32), Array(2.26671, dtype=float32), Array(2.3043315, dtype=float32), Array(2.5026038, dtype=float32), Array(1.9901841, dtype=float32), Array(2.5737429, dtype=float32), Array(2.6176026, dtype=float32), Array(2.1788547, dtype=float32), Array(2.519327, dtype=float32), Array(6.065675, dtype=float32), Array(2.0009763, dtype=float32), Array(1.9462322, dtype=float32)]
1 [Array(13.756277, dtype=float32), Array(2.1728497, dtype=float32), Array(2.7608593, dtype=float32), Array(2.3841074, dtype=float32), Array(30.37624, dtype=float32), Array(2.2532291, dtype=float32), Array(2.26671, dtype=float32), Array(2.3043315, dtype=float32), Array(2.5026038, dtype=float32), Array(1.9901841, dtype=float32), Array(2.5737429, dtype=float32), Array(2.6176026, dtype=float32), Array(2.1788547, dtype=float32), Array(2.5193

In [12]:
for j in range(len(rfes[1])):

    print(rfes[4][j],rfes[5][j])

13.758068 13.758068
2.1111844 2.1111844
2.7608593 2.7608593
2.4677556 2.4677556
11.372356 30.371368
2.2425961 2.2425961
2.265612 2.265612
2.2536604 2.2536604
2.480009 2.480009
2.154276 1.9865868
2.5762475 2.5762475
2.61777 2.61777
2.0800881 2.0800881
2.4883232 2.4883232
42.85688 6.063941
1.9224062 1.9224062
1.9108452 1.9108452


In [27]:
for i in range(1,9):
    path='./resultaxr/result'+str(i)+'.txt'
    sumt=0
    with open(path)as f:
        for j in f:
            xi=j.split()
            sumt+=float(xi[3][:-2])
            #print(xi[3][:-2])
    print(i-1,sumt/18)

0 91.53218187226173
1 90.94019914997942
2 88.88000077671411
3 88.95473814010609
4 83.84835663106699
5 89.11386164029426
6 89.10184460216088
7 88.19113530053022
